In [1]:
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pandas as pd
import demoji
from langdetect import detect
import re   # regular expression
import pandas as pd

In [2]:
CLIENT_SECRETS_FILE = "client_secret.json"

In [3]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

In [4]:
def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

In [5]:
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=993102206052-l8gds3r0jn9mk7f59kelrh1uog8bpa1n.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=rBOek8aePenNVgYxyXWe1h7k8qRaAs&prompt=consent&access_type=offline
Enter the authorization code: 4/2AHAJhTIH80ZUA_STC68DoCQ28NeSfnkVT-_Iz1JxwNpjeC7utcDG9g


In [6]:
query = "COVID-19"

In [7]:
query_results = service.search().list(part = 'snippet',q = query,
                                      order = 'relevance', 
                                      type = 'video',
                                      relevanceLanguage = 'en',
                                      safeSearch = 'moderate').execute()

In [8]:
query_results['items']

[{'kind': 'youtube#searchResult',
  'etag': 'G0bL2tmj-SIHSHgPF0k1g1Ol7KQ',
  'id': {'kind': 'youtube#video', 'videoId': 'H5UlTdyQTng'},
  'snippet': {'publishedAt': '2020-07-20T07:13:20Z',
   'channelId': 'UCCj956IF62FbT7Gouszaj9w',
   'title': 'Coronavirus vaccine: UK signs deals for 90 million doses - Covid-19: Top stories this morning - BBC',
   'description': 'Subscribe and to OFFICIAL BBC YouTube https://bit.ly/2IXqEIn Stream original BBC programmes FIRST on BBC iPlayer https://bbc.in/2J18jYJ ...',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/H5UlTdyQTng/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/H5UlTdyQTng/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/H5UlTdyQTng/hqdefault.jpg',
     'width': 480,
     'height': 360}},
   'channelTitle': 'BBC',
   'liveBroadcastContent': 'none',
   'publishTime': '2020-07-20T07:13:20Z'}},
 {'kind': 'youtube#searchResult',

In [9]:
video_id = []
channel = []
video_title = []
video_desc = []
for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])

In [10]:
video_id = video_id[0]
channel = channel[0]
video_title = video_title[0]
video_desc = video_desc[0]

In [11]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []


comments_temp = []
comment_id_temp = []
reply_count_temp = []
like_count_temp = []

In [12]:
nextPage_token = None

while 1:
  response = service.commentThreads().list(
                    part = 'snippet',
                    videoId = video_id,
                    maxResults = 100, 
                    order = 'relevance', 
                    textFormat = 'plainText',
                    pageToken = nextPage_token
                    ).execute()


  nextPage_token = response.get('nextPageToken')
  for item in response['items']:
      comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
      comment_id_temp.append(item['snippet']['topLevelComment']['id'])
      reply_count_temp.append(item['snippet']['totalReplyCount'])
      like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
      comments_pop.extend(comments_temp)
      comment_id_pop.extend(comment_id_temp)
      reply_count_pop.extend(reply_count_temp)
      like_count_pop.extend(like_count_temp)
        
      video_id_pop.extend([video_id]*len(comments_temp))
      channel_pop.extend([channel]*len(comments_temp))
      video_title_pop.extend([video_title]*len(comments_temp))
      video_desc_pop.extend([video_desc]*len(comments_temp))

  if nextPage_token is  None:
    break

# print(allVideos)

In [13]:
comments_pop

['Vaccines normally take like ten years to develop and this one there has been widely published failures while it has been rushed like the chimps that had a WORSE reaction to the virus.\n\nIf the government rushes 90 million doses and mandate it for people to work or travel and a significant cohort have adverse effects, will they have the money to compensate millions of people or will they be told to stuff it?\nThis is a serious question and would money ever be enough to assuage such a betrayal??\n\nJohn Adams and his wife in the USA braved the dangerous smallpox that was so dangerous and would kill entire bloodlines, that had an actual high IFR.\n\nThey did not force people to inoculate, they desperately tried it for themselves to try and survive.\n\nIf the government want to gain public trust they should follow that example, they should lead by example and first mandate that anyone who wants to sit in the House of Commons, which involves meetings many people, having meetings and clos

In [14]:
output_dict = {
        'Channel': channel_pop,
        'Video Title': video_title_pop,
        'Video Description': video_desc_pop,
        'Video ID': video_id_pop,
        'Comment': comments_pop,
        'Comment ID': comment_id_pop,
        'Replies': reply_count_pop,
        'Likes': like_count_pop,
        }

output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

In [15]:
output_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19503 entries, 0 to 19502
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Channel            19503 non-null  object
 1   Video Title        19503 non-null  object
 2   Video Description  19503 non-null  object
 3   Video ID           19503 non-null  object
 4   Comment            19503 non-null  object
 5   Comment ID         19503 non-null  object
 6   Replies            19503 non-null  int64 
 7   Likes              19503 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 1.2+ MB


In [16]:
output_df.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,Vaccines normally take like ten years to devel...,UgybqYhDdQFBecjhQBN4AaABAg,1,11
1,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,Vaccines normally take like ten years to devel...,UgybqYhDdQFBecjhQBN4AaABAg,1,11
2,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,Serious question: will this vaccine last beyon...,UgwOKUlYa2q5B2TbC-F4AaABAg,7,16
3,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,Vaccines normally take like ten years to devel...,UgybqYhDdQFBecjhQBN4AaABAg,1,11
4,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,Serious question: will this vaccine last beyon...,UgwOKUlYa2q5B2TbC-F4AaABAg,7,16


In [17]:
duplicates = output_df[output_df.duplicated("Comment ID")]
print("Count of duplicate comments in dataframe", duplicates.shape[0])

Count of duplicate comments in dataframe 19306


In [18]:
print("Count of unique comments in dataframe ",output_df.shape[0]- duplicates.shape[0])

Count of unique comments in dataframe  197


In [19]:
unique_df = output_df.drop_duplicates(subset=['Comment'])

In [20]:
unique_df.shape

(197, 8)

In [21]:
unique_df.head()

,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,Vaccines normally take like ten years to devel...,UgybqYhDdQFBecjhQBN4AaABAg,1,11
2,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,Serious question: will this vaccine last beyon...,UgwOKUlYa2q5B2TbC-F4AaABAg,7,16
5,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,why 90 million when there is only 66 million p...,Ugzw23so9Z7_RrRYyMl4AaABAg,8,20
9,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,How is this even possible when there has not e...,Ugy5uxxtnhyCltlyvxV4AaABAg,50,54
14,BBC,Coronavirus vaccine: UK signs deals for 90 mil...,Subscribe and to OFFICIAL BBC YouTube https://...,H5UlTdyQTng,That's a lot of shots for boris!,UgydkcxR4iZ74QAySlt4AaABAg,3,17


In [22]:
unique_df.to_csv("extracted_comments.csv",index = False)